# World Happiness and Economic Freedom

In [1]:
import pandas as pd
import dwfunctions as dw
import requests
import json
from pandas.io.json import json_normalize

In [2]:
ef_2019_raw = pd.read_csv('economic_freedom_index2019.csv', encoding='latin-1')
dw.strip_column(ef_2019_raw, 'Country')
ef_2019_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 34 columns):
CountryID                      186 non-null int64
Country Name                   186 non-null object
WEBNAME                        186 non-null object
Region                         186 non-null object
World Rank                     180 non-null float64
Region Rank                    180 non-null float64
2019 Score                     180 non-null float64
Property Rights                185 non-null float64
Judical Effectiveness          185 non-null float64
Government Integrity           185 non-null float64
Tax Burden                     180 non-null float64
Gov't Spending                 183 non-null float64
Fiscal Health                  183 non-null float64
Business Freedom               185 non-null float64
Labor Freedom                  184 non-null float64
Monetary Freedom               184 non-null float64
Trade Freedom                  182 non-null float64
Investment Freed

In [3]:
ef_2019_raw.head()

,CountryID,Country Name,WEBNAME,Region,World Rank,Region Rank,2019 Score,Property Rights,Judical Effectiveness,Government Integrity,...,Country,Population (Millions),"GDP (Billions, PPP)",GDP Growth Rate (%),5 Year GDP Growth Rate (%),GDP per Capita (PPP),Unemployment (%),Inflation (%),FDI Inflow (Millions),Public Debt (% of GDP)
0,1,Afghanistan,Afghanistan,Asia-Pacific,152.0,39.0,51.5,19.6,29.6,25.2,...,Afghanistan,35.5,$69.6,2.5,2.9,"$1,958",8.8,5.0,53.9,7.3
1,2,Albania,Albania,Europe,52.0,27.0,66.5,54.8,30.6,40.4,...,Albania,2.9,$36.0,3.9,2.5,"$12,507",13.9,2.0,"1,119.1",71.2
2,3,Algeria,Algeria,Middle East and North Africa,171.0,14.0,46.2,31.6,36.2,28.9,...,Algeria,41.5,$632.9,2.0,3.1,"$15,237",10.0,5.6,"1,203.0",25.8
3,4,Angola,Angola,Sub-Saharan Africa,156.0,33.0,50.6,35.9,26.6,20.5,...,Angola,28.2,$190.3,0.7,2.9,"$6,753",8.2,31.7,"-2,254.5",65.3
4,5,Argentina,Argentina,Americas,148.0,26.0,52.2,47.8,44.5,33.5,...,Argentina,44.1,$920.2,2.9,0.7,"$20,876",8.7,25.7,"11,857.0",52.6


In [4]:
ef_2019 = ef_2019_raw[['Country','2019 Score']].rename(columns={'2019 Score':'Economic Freedom Score'}).sort_values(by=['Country'])

ef_2019.head()

,Country,Economic Freedom Score
0,Afghanistan,51.5
1,Albania,66.5
2,Algeria,46.2
3,Angola,50.6
4,Argentina,52.2


In [5]:
#2019 World Happiness Data is the 4th <table> element on page
wh_2019_raw = pd.read_html('https://en.wikipedia.org/wiki/World_Happiness_Report')[4]
dw.strip_column(wh_2019_raw, 'Country or region')
wh_2019_raw.head()

,Overall rank,Country or region,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
0,1,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393
1,2,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410
2,3,Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341
3,4,Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118
4,5,Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298


In [6]:
wh_2019 = wh_2019_raw[['Country or region','Score']].rename(columns={'Country or region':'Country', 'Score':'Happiness Score'}).sort_values(by=['Country']).reset_index(drop=True)
wh_2019.head()

,Country,Happiness Score
0,Afghanistan,3.203
1,Albania,4.719
2,Algeria,5.211
3,Argentina,6.086
4,Armenia,4.559


In [7]:
ef_2019.info()
wh_2019.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186 entries, 0 to 185
Data columns (total 2 columns):
Country                   186 non-null object
Economic Freedom Score    180 non-null float64
dtypes: float64(1), object(1)
memory usage: 4.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 2 columns):
Country            151 non-null object
Happiness Score    151 non-null float64
dtypes: float64(1), object(1)
memory usage: 2.5+ KB


In [8]:
wh_countries = dw.get_countries(wh_2019)
ef_countries = dw.get_countries(ef_2019)

dw.diff_df(wh_countries, ef_countries, 'WH', 'EF', False)

Country
WH 30   Congo (Brazzaville)
   31      Congo (Kinshasa)
   34               Czechia
   63           Ivory Coast
   70            Kyrgyzstan
...                     ...
EF 166          Timor-Leste
   168                Tonga
   169  Trinidad and Tobago
   177        United States
   180              Vanuatu

[63 rows x 1 columns]

In [9]:
newCountryName = {37:wh_countries.loc[31,'Country'], 38:wh_countries.loc[30,'Country'], 
                  72:wh_countries.loc[55,'Country'], 40:wh_countries.loc[65,'Country'],
                  92:wh_countries.loc[73,'Country'], 93:wh_countries.loc[74,'Country'],
                  25:wh_countries.loc[95,'Country'], 103:wh_countries.loc[103,'Country'],
                  150:wh_countries.loc[123,'Country'], 89:wh_countries.loc[127,'Country'],
                  157:wh_countries.loc[128,'Country'], 55:wh_countries.loc[131,'Country']
                 }
for key, value in newCountryName.items():
    dw.change_name(ef_2019, key, 'Country', value) 

In [10]:
wh_countries = dw.get_countries(wh_2019)
ef_countries = dw.get_countries(ef_2019)

In [11]:
discrepencies = dw.diff_df(wh_countries, ef_countries, 'WH', 'EF', False)

In [12]:
wh_2019 = dw.remove_country(discrepencies, 'WH', wh_2019)
ef_2019 = dw.remove_country(discrepencies, 'EF', ef_2019)

In [13]:
wh_countries = dw.get_countries(wh_2019)
ef_countries = dw.get_countries(ef_2019)
dw.diff_df(wh_countries, ef_countries, 'WH', 'EF', False)

,,Country


In [14]:
wh_ef = dw.merge_df(wh_2019, ef_2019, 'Country', 'left')
wh_ef.info()
wh_ef.head(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149 entries, 0 to 148
Data columns (total 3 columns):
Country                   149 non-null object
Happiness Score           149 non-null float64
Economic Freedom Score    144 non-null float64
dtypes: float64(2), object(1)
memory usage: 4.7+ KB


,Country,Happiness Score,Economic Freedom Score
0,Afghanistan,3.203,51.5
1,Albania,4.719,66.5
2,Algeria,5.211,46.2
3,Argentina,6.086,52.2
4,Armenia,4.559,67.7
5,Australia,7.228,80.9
6,Austria,7.246,72.0
7,Azerbaijan,5.208,65.4
8,Bahrain,6.199,66.4
9,Bangladesh,4.456,55.6


In [15]:
wh_ef_2019 = wh_ef.dropna()
wh_ef_2019.info()
wh_ef_2019.head(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144 entries, 0 to 148
Data columns (total 3 columns):
Country                   144 non-null object
Happiness Score           144 non-null float64
Economic Freedom Score    144 non-null float64
dtypes: float64(2), object(1)
memory usage: 4.5+ KB


,Country,Happiness Score,Economic Freedom Score
0,Afghanistan,3.203,51.5
1,Albania,4.719,66.5
2,Algeria,5.211,46.2
3,Argentina,6.086,52.2
4,Armenia,4.559,67.7
5,Australia,7.228,80.9
6,Austria,7.246,72.0
7,Azerbaijan,5.208,65.4
8,Bahrain,6.199,66.4
9,Bangladesh,4.456,55.6


In [16]:
#Remove '#' to save .csv file to local drive
#wh_ef_2019.to_csv('wh_ef_final.csv', index=False)